In [52]:
from nornir import InitNornir
import pynetbox
import ast
from pprint import pprint
from global_var import (templates_path)
from jinja2 import Environment, FileSystemLoader

In [53]:
# Init Nornir
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.226:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

In [54]:
# Init pynetbox
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [55]:
changes = [
    {
        'id': 606, 
        'mtu': 1500,  
        'duplex': 'auto', 
        'description': 'T-SIDOROV', 
        'mode': 'access', 
        'untagged_vlan': 8
     }
]

nb.dcim.interfaces.update(changes)

[Ethernet 0]

In [56]:
#network_device = 136
user_device = 606
interface = ['mtu','mac_address','speed','duplex','description','mode','untagged_vlan']
#get_network_device = list(nb.dcim.interfaces.get(network_device)) # по ID находим интерфейс в netbox
get_user_device = nb.dcim.interfaces.get(user_device)

if get_user_device['connected_endpoints_reachable']:
    network_device = get_user_device['connected_endpoints'][0]['id']
    #data = []
    changes = dict()
    change_key = ['id']
    new_value = [network_device]

    for value in get_user_device:
        if value[0] in interface and value[1] != '' and value[1] != None:
        #if value[0] in interface:
            change_key.append(value[0])
            if isinstance(value[1], dict):
                new_value.append(list(value[1].values())[0])
            else:
                new_value.append(value[1])
    changes = [dict(zip(change_key,new_value))]

    print(changes)
#nb.dcim.interfaces.update(changes)

In [57]:
def check_connection_device(device_intf_id):
    """
    Проверяем, есть ли соединение у интерфейса
    Возвращаем словарь
    :param device_intf_id: ссылка на объект интерфейса pynetbox
    :return: device
    """
    intf = nb.dcim.interfaces.get(device_intf_id)
    device_keys = ['role','device_name','device_intf_name','neighbor'] # список ключей для словаря devices
    device = {}
    #pprint(dict(intf))
    if intf: # если интерфей существует
        #evices_keys = ['role','device_name','device_intf_name'] # список ключей для словаря devices
        device_values = []
        #-> Проверяем на наличие подключения
        if intf['connected_endpoints_reachable']:
            device_values.append(intf.device.device_role.slug) # роль устройства
            device_values.append(intf.device.name) # имя устройства
            device_values.append(intf.name) # имя интерфейса устройчтва
            device_values.append(intf.connected_endpoints[0].device) # устройство-сосед
        else:
            device_values.append(intf.device.device_role.slug) # роль устройства
            device_values.append(intf.device.name) # имя устройства
            device_values.append(intf.name) # имя интерфейса устройчтва
            device_values.append(None) # устройство-сосед
        #-<
        device = dict(zip(device_keys,device_values)) # получаем словарь
        #print(device)
        return device
    
    else: 
        print('No interface with the ID #{}'.format(device_intf_id))
        return None

In [58]:
def template_fill(a_device_id,a_device_type,b_object_id,b_device_type,status):
    
    template_file = "netbox_cable.template"
    environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
    template = environment.get_template(template_file)
    #print(a_device_id,b_object_id,status)
    content = None
    try:
        content = template.render( # заполняем шаблон
                                    a_object_id = a_device_id,
                                    a_object_type = a_device_type,
                                    b_object_id = b_object_id,
                                    b_object_type = b_device_type,
                                    status = status
                                )
    except: 
        # > добавляем запись в журнал
        comment,level =  'Not enough data to fill out the template','warning'                
        #print(journal_template_fill(comment,level,global_id,global_dcim))
        # <
    else:
        #print("Filling in the template...\n{}".format(content)) 
        print("Filling in the template...")
    
    return content

In [80]:
def nb_cable_conn():
    
    templates_roles = ['access_switch', 'user_device'] # присваиваем значение из netbox ("произвольные" данные)
    device_roles = []
    a_device_intf_id = 135 # мы точно знаем порт коммутатора (136)
    b_device_intf_id = 606 # выбираем из списка доступных (606)
    a_device_type = b_device_type = 'dcim.interface'
    a_device = check_connection_device(a_device_intf_id)
    b_device = check_connection_device(b_device_intf_id)

    if a_device and b_device:
        device_roles.append(a_device['role']) # заполняем список ролей
        device_roles.append(b_device['role'])
        
    if set(device_roles) == set(templates_roles): # проверяем, что получили устройства с разными ролями и соответствущие списку
        
        if a_device['neighbor'] and b_device['neighbor']: # если оба устройства получены
            # Будем ли удалять подключение (cable)?
            print('Оба устройства имеют подключение')
        elif a_device['neighbor']:
            # Будем ли удалять подключение (cable)?
            print('Интерфейс "{}" устройства "{}" подключен к другому устройству'.format(a_device['device_intf_name'],a_device['device_name']))
        elif b_device['neighbor']:
            # Будем ли удалять (cable)?
            print('Интерфейс "{}" устройства "{}" подключен к другому устройству'.format(b_device['device_intf_name'],b_device['device_name']))
        else: # оба устройства не имеют подключений
            content = ast.literal_eval(template_fill(a_device_intf_id,a_device_type,b_device_intf_id,b_device_type,status = 'connected'))
            print('Оба устройства не имеют подключений, создаем соединение')
            # POST cable
            nb.dcim.cables.create(content) # выполняем подключение (создание cable)
    #-> Проверяем роли устройств, т.к. соеднинение должно быть
    #   между конечным пользовательским устройством и коммутатором доступа
    
    #-<
    
    #-> Удаляем текущее подключение
    
    #-<
    
    return None

nb_cable_conn()

Оба устройства имеют подключение
